# Instalando Bibliotecas

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

# Carregando o dataset
É necessário realizar upload no mesmo diretório do script.

In [ ]:
import pandas as pd
df_train = pd.read_excel('/content/drive/MyDrive/Datasets/ABSA-HIN-ENSEMBLE/restaurants_train.xls');
df_train

,review,aspect,sentiment
0,But the staff was so horrible to us .,staff,-1
1,"To be completely fair , the only redeeming fac...",food,1
2,"The food is uniformly exceptional , with a ver...",food,1
3,"The food is uniformly exceptional , with a ver...",kitchen,1
4,"The food is uniformly exceptional , with a ver...",menu,0
...,...,...,...
3603,Each table has a pot of boiling water sunken i...,pot of boiling water,0
3604,Each table has a pot of boiling water sunken i...,meats,0
3605,Each table has a pot of boiling water sunken i...,vegetables,0
3606,Each table has a pot of boiling water sunken i...,rice,0


In [ ]:
df_test = pd.read_excel('/content/drive/MyDrive/Datasets/ABSA-HIN-ENSEMBLE/restaurants_test.xls');
df_test

,review,aspect,sentiment
0,The bread is top notch as well .,bread,1
1,I have to say they have one of the fastest del...,delivery times,1
2,Food is always fresh and hot - ready to eat !,Food,1
3,Did I mention that the coffee is OUTSTANDING ?,coffee,1
4,"Certainly not the best sushi in New York , how...",place,1
...,...,...,...
1115,"Creamy appetizers -- taramasalata , eggplant s...",Creamy appetizers,1
1116,"Creamy appetizers -- taramasalata , eggplant s...",warm pitas,0
1117,"Creamy appetizers -- taramasalata , eggplant s...",taramasalata,1
1118,"Creamy appetizers -- taramasalata , eggplant s...",eggplant salad,1


# Treinando o modelo

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 2.1MB 29.9MB/s 
     |████████████████████████████████| 1.2MB 30.8MB/s 
     |████████████████████████████████| 870kB 56.1MB/s 
     |████████████████████████████████| 3.3MB 54.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=fc7c6f5a04f9e756bf6f791f5a89143ae758211dd5fe3b42ca65b95d40873003
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=00a4c16763a5dd54f62a63da8e5642ce4e9e940b97a1cb8776851ea3d88c9d4c
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


In [ ]:
from sentence_transformers import SentenceTransformer, SentencesDataset, losses, CrossEncoder
from torch.utils.data import DataLoader
from sentence_transformers.readers import InputExample
model = CrossEncoder('microsoft/deberta-base',num_labels=3)
#model = CrossEncoder('microsoft/deberta-large',num_labels=3)

train_examples = []
for index,row in df_train.iterrows():
  x = InputExample(texts=[row['review'], row['aspect']], label=row['sentiment']+1)
  train_examples.append(x)

train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

model.fit(train_dataloader=train_dataloader,
          #evaluator=evaluator,
          epochs=5,
          warmup_steps=100)

Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['pooler.dense.weig

# Testando e avaliando

In [ ]:
train_test = []
for index,row in df_test.iterrows():
  x = [row['review'],row['aspect']]
  train_test.append(x)

preds = model.predict(train_test)
preds

array([[-2.8412821, -3.9733064,  5.9690146],
       [-3.0419512, -3.9094183,  6.1146684],
       [-3.127863 , -3.9455044,  6.1863704],
       ...,
       [-3.2072768, -3.812464 ,  6.234975 ],
       [-3.2213879, -3.8086221,  6.249223 ],
       [-3.2777956, -3.6863258,  6.213335 ]], dtype=float32)

In [ ]:
import numpy as np
y_pred = np.argmax(preds,axis=1)
y_true = np.array(df_test.sentiment.to_list())+1

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import numpy as np


f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
acc = accuracy_score(y_true, y_pred)

print(f1_micro,f1_macro,acc)

0.8821428571428571 0.8141214910576228 0.8821428571428571
